<a href="https://colab.research.google.com/github/sanozzz/First_app/blob/master/HighGainHighVol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import pandas as pd
import logging
from datetime import datetime
from pytz import timezone
import aiohttp
import asyncio
import nest_asyncio
import json
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import requests
from textwrap import fill
from google.colab import userdata
from openai import OpenAI
import re
import pytz
import openai
import time


# Allow nested event loops
nest_asyncio.apply()

# Set up the logger globally


def setup_logger():
    """Sets up a logger with both file and console handlers, formatted with IST timezone."""
    logger = logging.getLogger("HighGainHighVolLogger")

    # Avoid adding duplicate handlers
    if logger.hasHandlers():
        logger.handlers.clear()

    # Set the logging level
    logger.setLevel(logging.INFO)

    # Define a custom formatter with IST timezone
    class ISTFormatter(logging.Formatter):
        def formatTime(self, record, datefmt=None):
            ist = timezone("Asia/Kolkata")
            record_time = datetime.fromtimestamp(record.created, ist)
            return record_time.strftime(datefmt or "%Y-%m-%d %H:%M:%S")

    # Create the formatter
    formatter = ISTFormatter("%(asctime)s - %(levelname)s - %(message)s")

    # File handler setup
    try:
        log_file = f"/content/HighGainHighVol_{datetime.now(timezone('Asia/Kolkata')).strftime('%Y-%m-%d')}.log"
        file_handler = logging.FileHandler(log_file, mode="a")  # Append mode
        file_handler.setLevel(logging.INFO)
        file_handler.setFormatter(formatter)
        logger.addHandler(file_handler)
        print(f"File logging initialized at: {log_file}")
    except Exception as e:
        print(f"Failed to set up file handler: {e}")

    # Stream handler setup (console logging)
    try:
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        console_handler.setFormatter(formatter)
        logger.addHandler(console_handler)
        print("Console logging initialized.")
    except Exception as e:
        print(f"Failed to set up console handler: {e}")

    # Test the logger
    logger.info("Logger initialized successfully with IST timezone.")
    return logger


# Initialize the logger
logger = setup_logger()



def get_ist_timestamp():
    """Returns the current timestamp in IST."""
    ist = pytz.timezone('Asia/Kolkata')
    return datetime.now(ist).strftime('%Y-%m-%d %H:%M:%S')


class HighGainHighVol:
    def __init__(self, input_path='/content/HighGainHighVol.csv', output_path='/content/HighGainHighVol_with_DetailedSummaries.csv', plot_path='/content/HighGainHighVol.png'):
        """
        Initialize the Losers class with file paths and API keys for OpenAI and Perplexity.
        """
        self.input_path = input_path
        self.output_path = output_path
        self.plot_path = plot_path

        # Fetch API keys from Colab's userdata or environment variables
        openai_api_key = (userdata.get("OPENAI_API_KEY") or os.environ.get("OPENAI_API_KEY")).strip().replace("\r", "").replace("\n", "")
        perplexity_api_key = (userdata.get("PERPLEXITY_API_KEY") or os.environ.get("PERPLEXITY_API_KEY")).strip().replace("\r", "").replace("\n", "")


        if not openai_api_key:
            logger.error("OpenAI API key is not set. Please configure it in userdata or environment variables.")
            raise ValueError("OpenAI API key is missing.")
        if not perplexity_api_key:
            logger.error("Perplexity API key is not set. Please configure it in userdata or environment variables.")
            raise ValueError("Perplexity API key is missing.")

        # Initialize OpenAI client
        self.openai_api_key = openai_api_key
        self.initialize_openai_client()

        # Store the Perplexity API key for later use
        self.perplexity_api_key = perplexity_api_key
        self.api_url = "https://api.perplexity.ai/chat/completions"
        self.headers = {
            "Authorization": f"Bearer {self.perplexity_api_key}",
            "Content-Type": "application/json"
        }


        # Log successful initialization
        logger.info("TopLosers class initialized successfully.")

        # Validate the input file path
        self.validate_input_path()

    def initialize_openai_client(self):
        """Initialize the OpenAI client with the API key."""
        openai.api_key = self.openai_api_key
        self.client = openai
        logger.info("OpenAI client initialized successfully.")

    def validate_input_path(self):
        """Validate the existence of the input file."""
        if not os.path.exists(self.input_path):
            logger.error(f"Input file not found at: {self.input_path}")
            raise FileNotFoundError(f"Input file not found at: {self.input_path}")
        logger.info(f"Input file located at: {self.input_path}")

    def generate_contextual_summary(self, row, max_retries=3, retry_delay=2):
        """
        Generate a conversational summary for a given stock row using Perplexity's API.
        Retries the request in case of an error and processes blank columns for NewATL and NewATH.
        """
        # Handle missing or invalid stock column
        stock_column = "Stock" if "Stock" in row else "Stock Name"
        stock_name = row.get(stock_column, "Unknown").strip() if stock_column in row else "Unknown"

        # Handle blanks or NaN for NewATL and NewATH
        new_atl = row.get('NewATL', False)
        new_ath = row.get('NewATH', False)

        if pd.isna(new_atl):  # If blank or NaN, set to False
            new_atl = False
        if pd.isna(new_ath):  # If blank or NaN, set to False
            new_ath = False

        atl_commentary = (
            f"The stock hit a new all-time low today at ₹{row.get('Day Low', 'N/A')}, marking a significant decline."
            if new_atl and row.get('Day Low') is not None else ""
        )

        ath_commentary = (
            f"Previously, the stock reached an all-time high of ₹{row.get('Day High', 'N/A')}, reflecting its volatility."
            if new_ath and row.get('Day High') else ""
        )

        # Stellar volumes commentary
        stellar_volumes = (
            f"The stock displayed exceptional trading activity today with a daily turnover of ₹{row.get('DailyTurnInCr', 'N/A')} Cr, "
            f"trading {row.get('PercentFloatTradedPrimaryExchange', 0):.1f}% of its float on the primary exchange, "
            f"and a volume multiple of {row.get('Day volume multiple of week', 'N/A')} compared to the weekly average."
            if row.get('DailyTurnInCr', 0) > 100
            and row.get('PercentFloatTradedPrimaryExchange', 0) > 10
            and row.get('Day volume multiple of week', 0) > 2 else ""
        )

        # Performance summaries
        unstellar_quarterly_performance = (
            f"The stock has shown poor quarterly performance with a decline of {row.get('Qtr Change %', 0):.1f}%, "
            f"indicating challenges over the last quarter."
            if row.get('Qtr Change %', 0) < -15 else ""
        )

        stellar_quarterly_performance = (
            f"The stock has exhibited stellar quarterly performance with a growth of {row.get('Qtr Change %', 0):.1f}%, "
            f"indicating strong momentum over the last quarter."
            if row.get('Qtr Change %', 0) > 15 else ""
        )

        stellar_relative_performance = (
            f"The stock delivered stellar relative returns compared to Nifty500 this month ({row.get('Relative returns vs Nifty500 month%', 0):.1f}%) "
            f"and this quarter ({row.get('Relative returns vs Nifty500 quarter%', 0):.1f}%)."
            if row.get('Relative returns vs Nifty500 month%', 0) > 15 and row.get('Relative returns vs Nifty500 quarter%', 0) > 15 else ""
        )

        unstellar_relative_performance = (
            f"The stock underperformed significantly compared to Nifty500 this month ({row.get('Relative returns vs Nifty500 month%', 0):.1f}%) "
            f"and this quarter ({row.get('Relative returns vs Nifty500 quarter%', 0):.1f}%)."
            if row.get('Relative returns vs Nifty500 month%', 0) < -15 and row.get('Relative returns vs Nifty500 quarter%', 0) < -15 else ""
        )

        insider_trading_commentary = (
            f"Additionally, significant insider trading and SAST sales last quarter involved more than {row.get('Insider & SAST Buys Last Quarter', 0)} shares, "
            f"indicating a potential lack of confidence in the stock's future."
            if row.get('Insider & SAST Buys Last Quarter', 0) > 100000 else ""
        )

        notable_performance = " ".join(filter(None, [
            stellar_volumes,
            unstellar_quarterly_performance,
            stellar_quarterly_performance,
            stellar_relative_performance,
            unstellar_relative_performance,
            insider_trading_commentary,
            atl_commentary,
            ath_commentary
        ]))

        # Construct the query
        query = (
            f"Generate a summary for the following stock data:\n\n"
            f"- Stock: {stock_name}\n"
            f"- Day change: {row.get('Day change %', 0):.1f}%\n"
            f"- Closing price: ₹{row.get('Current Price', 'N/A')}\n"
            f"- Daily turnover: ₹{row.get('DailyTurnInCr', 'N/A')} Cr\n"
            f"- Trading volume multiple of the week: {row.get('Day volume multiple of week', 'N/A')}\n"
            f"- Percent of float traded on the primary exchange: {row.get('PercentFloatTradedPrimaryExchange', 0):.1f}%\n"
            f"- Revenue QoQ Growth: {row.get('Revenue QoQ Growth %', 0):.1f}%\n"
            f"- Basic EPS QoQ Growth: {row.get('Basic EPS QoQ Growth %', 0):.1f}%\n"
            f"- Day low: ₹{row.get('Day Low', 'N/A')}\n"
            f"- Day high: ₹{row.get('Day High', 'N/A')}\n"
            f"- New All-Time Low: {'Yes' if new_atl else 'No'}\n"
            f"- New All-Time High: {'Yes' if new_ath else 'No'}\n"
            f"{notable_performance}\n\n"
            f"Generate a friendly and engaging summary focusing on today's performance, recent decline, notable highs and lows, and overall trends.\n\n"
        )

        # Retry mechanism (unchanged)
        for attempt in range(max_retries):
            try:
                payload = {
                    "model": "llama-3.1-sonar-huge-128k-online",
                    "messages": [{"role": "user", "content": query}]
                }
                response = requests.post(
                    "https://api.perplexity.ai/chat/completions",
                    json=payload,
                    headers={"Authorization": f"Bearer {self.perplexity_api_key}", "Content-Type": "application/json"}
                )
                response.raise_for_status()
                summary = response.json().get("choices", [{}])[0].get("message", {}).get("content", "No summary generated.")
                logger.info(f"Perplexity summary generated for {stock_name}.")
                return summary
            except Exception as e:
                logger.error(f"Attempt {attempt + 1}/{max_retries} - Error generating summary for {stock_name}: {e}")
                if attempt < max_retries - 1:
                    logger.info(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    return f"Error generating summary after {max_retries} attempts: {e}"




    async def fetch_perplexity(self, session, query, retries=3, retry_delay=2):
        """
        Fetches data from Perplexity API with retry logic.

        Args:
            session: The aiohttp session for the request.
            query: The query to send to the Perplexity API.
            retries: Number of retries in case of failure.
            retry_delay: Delay in seconds between retries.

        Returns:
            dict: The response from the Perplexity API or an error message.
        """
        api_url = "https://api.perplexity.ai/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.perplexity_api_key}",
            "Content-Type": "application/json"
        }
        payload = {
            "model": "llama-3.1-sonar-huge-128k-online",
            "messages": [{"role": "user", "content": query}]
        }

        for attempt in range(retries):
            try:
                async with session.post(api_url, headers=headers, json=payload) as response:
                    if response.status == 200:
                        response_data = await response.json()
                        logger.info(f"Perplexity API response received for query: {query}")
                        return response_data
                    else:
                        logger.error(f"Perplexity API error: HTTP {response.status}")
                        if attempt < retries - 1:
                            logger.info(f"Retrying in {retry_delay} seconds...")
                            await asyncio.sleep(retry_delay)
                        else:
                            return {"error": f"HTTP {response.status}"}
            except Exception as e:
                logger.error(f"Error querying Perplexity API: {e}")
                if attempt < retries - 1:
                    logger.info(f"Retrying in {retry_delay} seconds...")
                    await asyncio.sleep(retry_delay)
                else:
                    return {"error": str(e)}

    def fetch_perplexity_results(self, queries, retries=3, retry_delay=2):
        """
        Fetches Perplexity news results for the given queries with retry logic.

        Args:
            queries: List of queries to fetch results for.
            retries: Number of retries in case of failure.
            retry_delay: Delay in seconds between retries.

        Returns:
            list: List of responses for each query.
        """
        async def fetch_all():
            async with aiohttp.ClientSession() as session:
                tasks = [
                    self.fetch_perplexity(session, query, retries=retries, retry_delay=retry_delay)
                    for query in queries
                ]
                return await asyncio.gather(*tasks)

        return asyncio.run(fetch_all())

    async def fetch_all_news(self, queries, retries=3, retry_delay=2):
        """
        Fetches all news results using Perplexity API with retry logic.

        Args:
            queries: List of queries to fetch results for.
            retries: Number of retries in case of failure.
            retry_delay: Delay in seconds between retries.

        Returns:
            list: List of responses for each query.
        """
        async with aiohttp.ClientSession() as session:
            tasks = [
                self.fetch_perplexity(session, query, retries=retries, retry_delay=retry_delay)
                for query in queries
            ]
            return await asyncio.gather(*tasks)

    def combine_summaries(self, row):
        """
        Generate a single, coherent, and engaging news byte from contextual and news summaries.
        Focuses on high gain and high volume stocks across the market, with a priority on price-impacting factors.
        """
        try:
            from datetime import datetime
            from pytz import timezone
            import requests
            import re

            # Determine market status based on IST
            ist = timezone("Asia/Kolkata")
            current_time = datetime.now(ist)
            market_close_time = current_time.replace(hour=15, minute=30, second=0, microsecond=0)

            market_status_message = (
                "This commentary is being generated while the market is open. "
                "Here are the highlights of high gain and high volume counters in the market today."
                if current_time < market_close_time else
                "This commentary is being generated after the market has closed. "
                "Here's a recap of today's high gain and high volume counters in the market."
            )

            # Extract stock information and summaries
            stock = row.get("Stock") or row.get("Stock Name", "Unknown Stock").strip()
            day_change = f"{row.get('Day change %', 'N/A')}%"
            closing_price = f"{row.get('Current Price', 'N/A')} rupees"
            turnover = f"{row.get('DailyTurnInCr', 'N/A')} crore rupees"
            volume_multiple = f"{row.get('Day volume multiple of week', 'N/A')} times the weekly average"
            float_traded = f"{row.get('PercentFloatTradedPrimaryExchange', 'N/A')}% of the float traded"

            contextual_summary = row.get('ContextualSummary', 'No contextual summary available.').strip()
            perplexity_news = row.get('PerplexityNews', 'No news available.').strip()

            # Truncate long summaries
            contextual_summary = (contextual_summary[:1000] + "...") if len(contextual_summary) > 1000 else contextual_summary
            perplexity_news = (perplexity_news[:1000] + "...") if len(perplexity_news) > 1000 else perplexity_news

            # Adjust commentary to remove unnecessary statements and correct factual inconsistencies
            commentary_lines = [
                f"{stock} registered a daily change of {day_change}, closing at {closing_price}.",
                f"The stock saw a significant turnover of {turnover}, with trading volume reaching {volume_multiple}.",
                f"Additionally, {float_traded} indicates notable investor activity."
            ]

            if row.get('NewATL', False):
                commentary_lines.append(f"The stock touched a new all-time low of {row.get('Day Low', 'N/A')} rupees today.")
            if row.get('NewATH', False):
                commentary_lines.append(f"It also hit a new all-time high of {row.get('Day High', 'N/A')} rupees today.")

            # Construct the input for the news byte
            combined_input = (
                f"{market_status_message}\n\n"
                f"\n".join(commentary_lines) + "\n\n"
                f"### Contextual Highlights:\n"
                f"{contextual_summary}\n\n"
                f"### News Highlights:\n"
                f"{perplexity_news}\n\n"
                f"### Instructions for News Byte Generation:\n"
                f"Craft an engaging and concise news byte based on the provided details. The output should:\n"
                f"- **Highlight Volumes and Turnover:** Focus on significant trading volumes, turnover, and float traded percentages.\n"
                f"- **Analyze Price Impact:** Connect price changes to key metrics like revenue growth, EPS, partnerships, or other impactful events.\n"
                f"- **Emphasize Investor Sentiment:** Discuss how market sentiment influenced the stock's performance today.\n"
                f"- **Provide Context:** Relate the movement to specific events or announcements.\n\n"
                f"### Output Requirements:\n"
                f"- Use a conversational tone and ensure clarity for a general audience.\n"
                f"- Avoid markdowns, citations, or links.\n"
                f"- Expand abbreviations and replace '₹' with 'rupees.'\n"
                f"- Limit to under 500 words for suitability in a 2-3 minute video format.\n"
                f"End with a note inviting users to stay tuned for more market updates."
            )

            # API call to generate the summary
            payload = {
                "model": "llama-3.1-sonar-huge-128k-online",
                "messages": [{"role": "user", "content": combined_input}]
            }
            headers = {
                "Authorization": f"Bearer {self.perplexity_api_key}",
                "Content-Type": "application/json"
            }

            response = requests.post("https://api.perplexity.ai/chat/completions", json=payload, headers=headers)

            if response.status_code == 200:
                result = response.json()
                summary = result.get("choices", [{}])[0].get("message", {}).get("content", "No summary generated.")
                return re.sub(r"\[\d+\]", "", summary).strip()  # Clean markdown-like citations
            else:
                logger.error(f"API Error: {response.status_code} - {response.text}")
                return f"Error generating summary: {response.status_code}"

        except Exception as e:
            logger.error(f"Error in combine_summaries: {e}")
            return f"Error combining summaries: {e}"


    def generate_initial_news_byte(self, df, retries=3, retry_delay=2):
        """
        Generates the initial news byte from combined summaries and daily changes with retry logic.
        Focuses on high gain and high volume stocks across the market, emphasizing Day Change % and turnover.
        """
        try:
            # Determine the stock column dynamically
            stock_column = "Stock" if "Stock" in df.columns else "Stock Name"
            if stock_column not in df.columns:
                logger.error("Neither 'Stock' nor 'Stock Name' column found in the DataFrame.")
                return "Error: Required column 'Stock' or 'Stock Name' not found."

            # Prepare the stock changes with high gain and high volume context
            stock_changes = "\n".join(
                f"{row.get(stock_column, 'Unknown Stock')} saw a gain of {row.get('Day change %', 'N/A')}% "
                f"with a turnover of ₹{row.get('DailyTurnInCr', 'N/A')} crore and {row.get('PercentFloatTradedPrimaryExchange', 'N/A')}% of its float traded."
                for _, row in df.iterrows()
            )
            logger.debug(f"Stock changes prepared: {stock_changes}")

            # Determine market status
            ist = timezone("Asia/Kolkata")
            current_time = datetime.now(ist)
            market_close_time = current_time.replace(hour=15, minute=30, second=0, microsecond=0)

            market_status_message = (
                "This commentary is being generated while the market is open. "
                "Here are today's high gain and high volume stocks across the market."
                if current_time < market_close_time else
                "This commentary is being generated after the market has closed. "
                "Here's a summary of today's high gain and high volume stocks across the market."
            )
            logger.debug(f"Market status message prepared: {market_status_message}")

            combined_input = (
                f"Generate a compelling and engaging news byte summarizing today's high gain and high volume stocks across the market. "
                f"Focus on significant trading metrics, emphasizing Day Change %, turnover, and float traded.\n\n"
                f"Stocks: {', '.join(df[stock_column])}\n\n"
                f"Performance Summary:\n{stock_changes}\n\n"
                f"Market Status:\n{market_status_message}\n\n"
                f"Structure the response as a seamless narrative, adhering to these guidelines:\n\n"
                f"1. **Stock-Specific Highlights:**\n"
                f"   - Explicitly mention each stock name followed by its Day Change %, turnover, and float traded. Example: 'Reliance Industries saw a gain of 2.5% with a turnover of ₹500 crore and 12% of its float traded.'\n\n"
                f"2. **Analyze Key Metrics:**\n"
                f"   - Explain notable metrics like revenue growth, EPS, or partnerships that impacted performance.\n"
                f"   - Example: 'Reliance Industries' performance was driven by a 20% surge in quarterly revenue, following its expansion into renewable energy.'\n\n"
                f"3. **Provide Market Context:**\n"
                f"   - Highlight any industry trends, macroeconomic factors, or global developments influencing the stock's movement.\n"
                f"   - Example: 'The broader market sentiment improved on hopes of an interest rate pause, boosting large-cap stocks.'\n\n"
                f"4. **Formatting and Tone:**\n"
                f"   - Avoid bullet points in the output. Write in a cohesive narrative style.\n"
                f"   - Use full forms like 'Limited' instead of 'Ltd.' and replace '₹' with 'rupees.'\n\n"
                f"5. **Conciseness and Engagement:**\n"
                f"   - Keep the response under 500 words, ensuring it fits a 2-3 minute video format.\n"
                f"   - Maintain an accessible, conversational tone throughout.\n\n"
                f"Deliver the response as a polished and engaging news byte with each stock's performance and context comprehensively covered."
            )

            # Payload for the Perplexity API
            payload = {
                "model": "llama-3.1-sonar-huge-128k-online",
                "messages": [{"role": "user", "content": combined_input}],
                "max_tokens": 1500
            }

            # Retry logic for API requests
            for attempt in range(retries):
                try:
                    response = requests.post(self.api_url, json=payload, headers=self.headers)
                    if response.status_code == 200:
                        result = response.json()
                        news_byte = result.get('choices', [{}])[0].get('message', {}).get('content', "No news byte generated.")

                        # Clean the generated news byte
                        cleaned_news_byte = re.sub(r"\[\d+\]", "", news_byte)  # Remove citations like [1], [2]
                        cleaned_news_byte = re.sub(r"\*\*.*?\*\*", "", cleaned_news_byte).strip()  # Remove markdown formatting
                        logger.info("Initial news byte generated successfully.")
                        return cleaned_news_byte
                    else:
                        logger.error(f"Perplexity API error: {response.status_code} - {response.text}")
                        if attempt < retries - 1:
                            logger.info(f"Retrying in {retry_delay} seconds...")
                            time.sleep(retry_delay)
                        else:
                            return f"Error generating initial news byte: {response.status_code}"
                except Exception as e:
                    logger.error(f"Exception generating initial news byte on attempt {attempt + 1}: {e}")
                    if attempt < retries - 1:
                        logger.info(f"Retrying in {retry_delay} seconds...")
                        time.sleep(retry_delay)
                    else:
                        return f"Error generating initial news byte: {e}"
        except Exception as e:
            logger.error(f"Unexpected error in generate_initial_news_byte: {e}")
            return f"Error generating initial news byte: {e}"



    def shorten_news_byte(self, initial_news_byte, retries=3, retry_delay=2):
        """
        Shortens the news byte if it exceeds 500 words, with retry logic.
        Ensures that stock names and high gain/high volume updates are preserved in the shortened version.
        """
        shorten_request = (
            "The previous response exceeded 500 words. Please provide a shorter version "
            "of the following news byte, strictly under 500 words while retaining all key information. "
            "Focus specifically on high gain and high volume stocks. It is essential to preserve the stock names and "
            "explicitly mention their performance metrics such as Day Change %, turnover, and float traded. "
            "For example, include phrases like 'Reliance Industries gained 2.5% with a turnover of rupees 500 crore' "
            "or 'Tata Steel surged 3.2% trading 15% of its float.' Each stock name must be clearly mentioned.\n\n"
            f"{initial_news_byte}\n\n"
            "Key Instructions:\n"
            "- Retain all stock names and explicitly mention their high gain/high volume performance.\n"
            "- Include critical metrics such as turnover, float traded, or relevant news that explains the movement of each stock.\n"
            "- Avoid generalizations or omitting stock-specific details.\n"
            "- Use engaging and conversational language suitable for a 2-3 minute video format.\n"
            "- Remove redundant phrases and ensure the response is concise.\n"
            "- Avoid citations (e.g., [1], [2]) or markdown-style formatting (e.g., '**text**').\n"
            "- Replace symbols like '₹' with 'rupees' and expand 'Ltd.' to 'Limited.'\n"
            "- Ensure numerical ranges (e.g., 80-100) are written as '80 to 100.'\n"
            "- Ensure all content is under 500 words and easy to understand."
        )

        payload = {
            "model": "llama-3.1-sonar-huge-128k-online",
            "messages": [{"role": "user", "content": shorten_request}],
            "max_tokens": 1500
        }

        # Retry logic
        for attempt in range(retries):
            try:
                response = requests.post(self.api_url, json=payload, headers=self.headers)
                if response.status_code == 200:
                    result = response.json()
                    news_byte = result.get('choices', [{}])[0].get('message', {}).get('content', "No news byte generated.")

                    # Clean the shortened news byte
                    cleaned_news_byte = re.sub(r"\*\*.*?\*\*", "", news_byte).strip()
                    logger.info("Shortened news byte generated successfully.")
                    return cleaned_news_byte
                else:
                    logger.error(f"Perplexity API error: {response.status_code} - {response.text}")
                    if attempt < retries - 1:
                        logger.info(f"Retrying in {retry_delay} seconds...")
                        time.sleep(retry_delay)
                    else:
                        return f"Error shortening news byte: {response.status_code}"
            except Exception as e:
                logger.error(f"Exception shortening news byte: {e}")
                if attempt < retries - 1:
                    logger.info(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    return f"Error shortening news byte: {e}"



    def generate_final_news_byte(self, df):
        """
        Generates the final news byte, shortening it if necessary.
        Focuses on high gain and high volume stocks while retaining stock names and key updates.
        Removes citations like [1], [2], and ensures a clean, concise output.
        Includes retries in case of errors and comprehensive logging.
        """
        try:
            # Ensure the stock column exists dynamically
            stock_column = "Stock" if "Stock" in df.columns else "Stock Name"
            if stock_column not in df.columns:
                logger.error("Neither 'Stock' nor 'Stock Name' column found in the DataFrame.")
                return "Error: Required column 'Stock' or 'Stock Name' not found."

            # Generate the initial news byte
            logger.info("Starting to generate the initial news byte.")
            initial_news_byte = self.generate_initial_news_byte(df)

            if not initial_news_byte or "Error" in initial_news_byte:
                logger.error("Failed to generate the initial news byte.")
                return "Error generating the initial news byte."

            # Clean citations from the initial news byte
            cleaned_initial_news_byte = re.sub(r"\[\d+\]", "", initial_news_byte)  # Remove citations like [1], [2]
            logger.info("Cleaned citations from the initial news byte.")

            # Calculate word count of the cleaned initial news byte
            word_count = len(cleaned_initial_news_byte.split())
            logger.info(f"Initial news byte generated with {word_count} words after cleaning.")

            # Define the word limit
            WORD_LIMIT = 500

            # If within the word limit, return the cleaned initial news byte
            if word_count <= WORD_LIMIT:
                logger.info("Initial news byte is within the word limit. Returning the final news byte.")
                return cleaned_initial_news_byte

            # Log and attempt to shorten the news byte
            logger.warning(f"Initial news byte exceeded {WORD_LIMIT} words ({word_count}). Attempting to shorten.")
            max_retries = 3
            retry_delay = 2  # Delay between retries in seconds
            shortened_news_byte = None

            for attempt in range(max_retries):
                try:
                    logger.info(f"Attempt {attempt + 1}/{max_retries} to shorten the news byte.")
                    shortened_news_byte = self.shorten_news_byte(cleaned_initial_news_byte)

                    # Clean citations from the shortened news byte
                    cleaned_shortened_news_byte = re.sub(r"\[\d+\]", "", shortened_news_byte)  # Remove citations
                    final_word_count = len(cleaned_shortened_news_byte.split())
                    logger.info(f"Shortened news byte generated with {final_word_count} words after cleaning.")

                    # Check if the shortened news byte is within the word limit
                    if final_word_count <= WORD_LIMIT:
                        logger.info("Shortened news byte is within the word limit. Returning the final news byte.")
                        return cleaned_shortened_news_byte
                    else:
                        logger.warning(f"Shortened news byte still exceeds the word limit ({final_word_count} words).")
                except Exception as e:
                    logger.error(f"Error shortening the news byte on attempt {attempt + 1}: {e}")
                    if attempt < max_retries - 1:
                        logger.info(f"Retrying in {retry_delay} seconds...")
                        time.sleep(retry_delay)

            # If retries are exhausted, return the cleaned initial news byte with a warning
            logger.warning("Failed to shorten the news byte within the word limit after multiple attempts.")
            return f"The news byte exceeds the word limit but here is the unshortened version:\n\n{cleaned_initial_news_byte}"

        except Exception as e:
            logger.error(f"Unexpected error in generating the final news byte: {e}")
            return "Error generating the final news byte."


    def refine_final_news_byte(self, df, retries=3, retry_delay=2):
        """
        Refines the `FinalNewsByte` column for stocks already present using ChatGPT API.

        Args:
            df (pd.DataFrame): The DataFrame containing the `FinalNewsByte` column.
            retries (int): Number of retries in case of API errors.
            retry_delay (int): Delay in seconds between retries.

        Returns:
            pd.DataFrame: Updated DataFrame with refined `FinalNewsByte`.
        """
        if 'FinalNewsByte' not in df.columns:
            logger.error("`FinalNewsByte` column not found in DataFrame.")
            return df

        if df['FinalNewsByte'].isna().all():
            logger.error("`FinalNewsByte` column is empty or contains only NaN values.")
            return df

        # Ensure only stocks present in the DataFrame are referenced
        stock_names = df['Stock'].dropna().unique().tolist()

        # Prepare the news byte for refinement
        final_news_byte = df['FinalNewsByte'].iloc[0]  # Assumes all rows share the same content

        if not isinstance(final_news_byte, str):
            logger.error("Invalid type for FinalNewsByte. Expected string.")
            df['FinalNewsByte'] = "Error: Invalid FinalNewsByte format."
            return df

        refine_request = (
            "Refine the following news byte about today's stock market updates to make it more engaging and professional, "
            "suitable for a financial audience. Emphasize key metrics, highlight significant trends, and ensure a conversational tone. "
            "Focus only on the stocks mentioned explicitly and avoid introducing additional stocks or irrelevant details:\n\n"
            f"Stocks: {', '.join(stock_names)}\n"
            f"{final_news_byte}\n\n"
            "Do not include markdown formatting or any instructions. Focus on providing a polished and publication-ready summary."
        )

        payload = {
            "model": "gpt-4",
            "messages": [{"role": "user", "content": refine_request}],
            "max_tokens": 1024
        }

        for attempt in range(retries):
            try:
                response = requests.post("https://api.openai.com/v1/chat/completions",
                                        json=payload,
                                        headers={"Authorization": f"Bearer {self.openai_api_key}", "Content-Type": "application/json"})

                if response.status_code == 200:
                    result = response.json()
                    refined_news_byte = result.get('choices', [{}])[0].get('message', {}).get('content', "No refinement generated.")
                    refined_news_byte = refined_news_byte.strip()
                    logger.info("News byte refined successfully.")
                    df['FinalNewsByte'] = refined_news_byte  # Update all rows with the refined content
                    return df
                else:
                    logger.error(f"ChatGPT API error: {response.status_code} - {response.text}")
                    if attempt < retries - 1:
                        logger.info(f"Retrying in {retry_delay} seconds...")
                        time.sleep(retry_delay)
                    else:
                        df['FinalNewsByte'] = "Error refining news byte."

            except Exception as e:
                logger.error(f"Exception during refinement: {e}")
                if attempt < retries - 1:
                    logger.info(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    df['FinalNewsByte'] = "Error refining news byte."

        return df








    def shorten_summary_for_reel(self, summary, max_tokens=500, retries=3, retry_delay=2):
        """
        Shortens the provided summary to fit within the token limit for a 1-minute reel.

        Args:
            summary (str): The input summary to be shortened.
            max_tokens (int): Maximum tokens allowed for the shortened summary.
            retries (int): Number of retry attempts in case of API errors.
            retry_delay (int): Delay (in seconds) between retries.

        Returns:
            str: The shortened summary.
        """
        logger.info("Shortening summary for 1-minute reel focused on high gain and high volume counters.")

        shorten_request = (
            "Please shorten the following summary to fit within a 1-minute video duration. "
            f"Limit the response to {max_tokens} tokens, retaining all key points, especially focusing on high gain and high volume stocks, and making it conversational:\n\n"
            f"{summary}\n\n"
            "Key Instructions:\n"
            "- Mention all stock names explicitly alongside their % Day Change, turnover, and float traded.\n"
            "- Retain critical updates like revenue growth, partnerships, or market sentiment linked to the stocks.\n"
            "- Avoid markdown formatting and citations like [1], [2].\n"
            "- Replace symbols like '₹' with 'rupees' and expand abbreviations such as 'Ltd.' to 'Limited.'\n"
            "- Ensure the language is concise, engaging, and suitable for a 1-minute reel format."
        )

        payload = {
            "model": "llama-3.1-sonar-huge-128k-online",
            "messages": [{"role": "user", "content": shorten_request}],
            "max_tokens": max_tokens
        }

        # Retry logic
        for attempt in range(retries):
            try:
                response = requests.post(self.api_url, json=payload, headers=self.headers)
                if response.status_code == 200:
                    result = response.json()
                    shortened_summary = result.get('choices', [{}])[0].get('message', {}).get('content', "No summary generated.")

                    # Clean the shortened summary
                    cleaned_summary = re.sub(r"\*\*.*?\*\*", "", shortened_summary).strip()
                    logger.info("Reel summary successfully shortened.")
                    return cleaned_summary
                else:
                    logger.error(f"Perplexity API error: {response.status_code} - {response.text}")
                    if attempt < retries - 1:
                        logger.info(f"Retrying in {retry_delay} seconds...")
                        time.sleep(retry_delay)
                    else:
                        return f"Error shortening summary: {response.status_code}"
            except Exception as e:
                logger.error(f"Exception shortening summary: {e}")
                if attempt < retries - 1:
                    logger.info(f"Retrying in {retry_delay} seconds...")
                    time.sleep(retry_delay)
                else:
                    return f"Error shortening summary: {e}"


    def generate_reel_byte(self, df):
        """
        Generates a 1-minute ReelByte for individual stocks and combined news, focusing on high gain and high volume updates.

        Args:
            df (pd.DataFrame): DataFrame containing stock data with summaries.

        Adds:
            ReelByteIndividual (list): 1-minute summaries for individual stocks.
            ReelByteCombined (str): 1-minute summary for the combined news.
        """
        try:
            logger.info("Generating 1-minute ReelBytes for high gain and high volume stocks.")

            # Check input type
            if not isinstance(df, pd.DataFrame):
                logger.error(f"Invalid input to generate_reel_byte: Expected DataFrame, got {type(df)}")
                return

            # Generate ReelByte for individual stocks
            reel_summaries = []
            for summary in df['CombinedSummary']:
                if not isinstance(summary, str):
                    logger.error(f"Invalid type for summary: {type(summary)}. Expected a string.")
                    reel_summaries.append("Error: Invalid summary format.")
                    continue

                try:
                    reel_summary = self.shorten_summary_for_reel(summary)
                    reel_summaries.append(reel_summary)
                except Exception as e:
                    logger.error(f"Error shortening summary for reel: {e}")
                    reel_summaries.append("Error: Could not generate ReelByte.")

            df['ReelByteIndividual'] = reel_summaries

            # Generate ReelByte for the combined news summary
            if 'FinalNewsByte' in df.columns:
                combined_news_summary = df['FinalNewsByte'].iloc[0]  # Assume all rows have identical FinalNewsByte
                if not isinstance(combined_news_summary, str):
                    logger.error(f"Invalid type for FinalNewsByte: {type(combined_news_summary)}. Expected a string.")
                    reel_combined_news = "Error: Invalid FinalNewsByte format."
                else:
                    try:
                        reel_combined_news = self.shorten_summary_for_reel(combined_news_summary)
                    except Exception as e:
                        logger.error(f"Error shortening combined news for reel: {e}")
                        reel_combined_news = "Error: Could not generate ReelByte."
            else:
                logger.error("FinalNewsByte column not found in DataFrame.")
                reel_combined_news = "Error: FinalNewsByte not found."

            df['ReelByteCombined'] = [reel_combined_news] * len(df)

            logger.info("1-minute ReelBytes successfully generated.")
        except Exception as e:
            logger.error(f"Error generating ReelByte: {e}")




    def upload_image_to_s3(self, file_path, title):
        """
        Uploads an image file to an S3 bucket and returns the public URL of the uploaded file.
        """
        try:
            aws_access_key_id = userdata.get("aws_access_key_id")
            aws_secret_access_key = userdata.get("aws_secret_access_key")

            if not aws_access_key_id or not aws_secret_access_key:
                raise Exception("AWS credentials are not set in Colab userdata.")

            # Create an S3 client
            s3 = boto3.client(
                "s3",
                aws_access_key_id=aws_access_key_id,
                aws_secret_access_key=aws_secret_access_key,
                region_name="ap-south-1",
            )

            # Get the current date and time in a file-safe format
            current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

            # Generate the S3 file key with datetime
            sanitized_title = "".join(e for e in title if e.isalnum() or e == "_").replace(" ", "_").lower()
            file_key = f"images/{sanitized_title}_{current_datetime}.png"

            # Upload the image file to the S3 bucket
            s3.upload_file(file_path, "finbytes", file_key)

            # Construct the public URL for the uploaded file
            file_url = f"https://finbytes.s3.ap-south-1.amazonaws.com/{file_key}"
            logger.info(f"Image uploaded to S3 successfully. URL: {file_url}")
            return file_url
        except Exception as e:
            logger.error(f"Failed to upload image to S3: {str(e)}")
            raise Exception(f"Failed to upload image to S3: {str(e)}")



    def generate_plot(self, df):
        try:
            # Dynamically determine the stock column
            stock_column = "Stock" if "Stock" in df.columns else "Stock Name"
            if stock_column not in df.columns:
                logger.error("Neither 'Stock' nor 'Stock Name' column found in the DataFrame.")
                return

            # Wrap long stock names for better display
            df[stock_column] = df[stock_column].apply(lambda x: fill(x, width=15))

            # Add a helper column for color coding
            df['Color'] = df['Day change %'].apply(lambda x: '#008000' if x > 0 else '#FF0000')  # Green for positive, Red for negative

            # Set the aesthetic style of the plots
            sns.set_theme(style="ticks")  # Use 'ticks' to enable control of spines

            # Create the figure with mobile-friendly dimensions
            plt.figure(figsize=(6, 10))  # Adjust aspect ratio (width x height)

            # Create the horizontal bar plot
            bars = sns.barplot(
                x='Day change %',
                y=stock_column,
                data=df,
                palette=df['Color'].tolist(),  # Apply precomputed colors
                orient='h'
            )

            # Add data labels on the bars
            for bar in bars.patches:
                bars.annotate(
                    f"{bar.get_width():.1f}%",  # Format to 1 decimal place
                    (bar.get_width() + 0.2, bar.get_y() + bar.get_height() / 2),  # Position the text
                    ha='left', va='center', fontsize=10, fontweight='bold', color='white', xytext=(5, 0),
                    textcoords='offset points'
                )

            # Remove gridlines and retain only left and bottom axes
            sns.despine(left=False, bottom=False)  # Retain left and bottom borders
            plt.gca().grid(False)  # Turn off all gridlines

            # Set the main title
            plt.title('Daily % Change of Stocks', fontsize=16, fontweight='bold', pad=15, color='white')

            # Set axis labels
            plt.xlabel('Daily % Change', fontsize=12, fontweight='bold', color='white')
            plt.ylabel('Stocks', fontsize=12, fontweight='bold', color='white')

            # Adjust font size and make stock names bold
            plt.xticks(fontsize=10, fontweight='bold', color='white')
            plt.yticks(fontsize=10, fontweight='bold', ha='right', color='white')  # Align stock names to the left

            # Set background color to black
            plt.gca().set_facecolor('black')
            plt.tight_layout()

            # Save the plot with black background
            plt.savefig(self.plot_path, dpi=200, facecolor='black')  # Ensure the background is saved as black
            plt.close()

            logger.info(f"Mobile-friendly horizontal bar plot saved to {self.plot_path}")
        except Exception as e:
            logger.error(f"Error generating horizontal bar plot: {e}")





    def save_summary_as_json(self, final_news_byte, plot_url):
        """
        Save the final combined news byte and metadata as a JSON file.
        """
        # Get current time in IST (UTC+5:30)
        ist = timezone("Asia/Kolkata")
        created_time = datetime.now(ist).isoformat()  # Generate ISO format with timezone

        # JSON output
        output = {
            "created_time": created_time,  # Use IST with UTC+5:30 offset
            "title": "High Gain & High Volumes Today",
            "summary_text": final_news_byte,
            "image_url": plot_url,  # Include the uploaded plot URL
        }

        # Save the JSON to a file
        json_path = "/content/HighGainHighVolume.json"
        try:
            with open(json_path, "w") as f:
                json.dump(output, f, indent=4)
            logger.info(f"JSON summary saved to {json_path}")
        except Exception as e:
            logger.error(f"Error saving JSON summary to {json_path}: {e}")


    def post_json_to_summary(self, json_data):
        """
        Posts a JSON summary to the specified API endpoint.

        Args:
            json_data (dict): The JSON data to be posted to the API.

        Returns:
            dict: The response from the API if successful.

        Raises:
            Exception: If the POST request fails.
        """
        api_url = "https://sy6foef31c.execute-api.ap-south-1.amazonaws.com/default/create"
        headers = {"Content-Type": "application/json"}

        try:
            response = requests.post(api_url, json=json_data, headers=headers)
            response.raise_for_status()  # Raise an error for non-2xx responses
            logger.info(f"POST request successful. Response: {response.json()}")
            return response.json()
        except Exception as e:
            logger.error(f"Error posting JSON to API: {e}")
            raise

    def run(self):
        """Main method to execute the Top Gainers and Losers process."""
        logger.info("Starting the Top Gainers and Losers process.")

        # Step 1: Read and preprocess the input CSV
        try:
            df = pd.read_csv(self.input_path)
        except FileNotFoundError:
            logger.error(f"Input file not found at {self.input_path}.")
            return
        except Exception as e:
            logger.error(f"Error reading input file: {e}")
            return

        # Validate DataFrame
        if not isinstance(df, pd.DataFrame):
            logger.error("Invalid input file. Expected a DataFrame.")
            return

        # Determine the column to use for stock names
        stock_column = None
        if "Stock" in df.columns:
            stock_column = "Stock"
        elif "Stock Name" in df.columns:
            stock_column = "Stock Name"

        if not stock_column:
            logger.error("Neither 'Stock' nor 'Stock Name' columns are present in the input file.")
            return

        # Ensure the necessary columns exist
        required_columns = [stock_column, "DailyTurnInCr"]
        missing_columns = set(required_columns) - set(df.columns)
        if missing_columns:
            logger.error(f"Missing required columns: {missing_columns}. Ensure the input file includes these columns.")
            return

        # Handle missing or invalid values in the DataFrame
        df[stock_column] = df[stock_column].fillna('Unknown')
        df['DailyTurnInCr'] = pd.to_numeric(df['DailyTurnInCr'], errors='coerce').fillna(0)

        # Step 2: Process top 4 stocks based on "DailyTurnInCr"
        logger.info("Selecting top 4 stocks by DailyTurnInCr.")
        try:
            df = df.nlargest(4, "DailyTurnInCr").sort_values(by="DailyTurnInCr", ascending=False)
        except Exception as e:
            logger.error(f"Error selecting top stocks by DailyTurnInCr: {e}")
            return

        # Step 3: Generate Contextual summaries
        logger.info("Generating Contextual summaries.")
        try:
            df['ContextualSummary'] = df.apply(self.generate_contextual_summary, axis=1)
        except Exception as e:
            logger.error(f"Error generating contextual summaries: {e}")
            df['ContextualSummary'] = ["Error generating summary" for _ in range(len(df))]

        # Step 4: Fetch Perplexity news
        logger.info("Fetching news from Perplexity.")
        try:
            perplexity_queries = [
                f"Find recent news and updates about {row[stock_column]} in the last week."
                for _, row in df.iterrows() if row[stock_column] != 'Unknown'
            ]
            perplexity_results = self.fetch_perplexity_results(perplexity_queries)
            df['PerplexityNews'] = [
                res.get("choices", [{"message": {"content": "No recent news found."}}])[0]["message"]["content"]
                if isinstance(res, dict) and "error" not in res else "Error occurred"
                for res in perplexity_results
            ]
        except Exception as e:
            logger.error(f"Error fetching news from Perplexity: {e}")
            df['PerplexityNews'] = ["Error fetching news" for _ in range(len(df))]

        # Step 5: Combine ChatGPT summaries with Perplexity news
        logger.info("Combining Contextual summaries and Perplexity news.")
        try:
            df['CombinedSummary'] = df.apply(self.combine_summaries, axis=1)
        except Exception as e:
            logger.error(f"Error combining summaries: {e}")
            df['CombinedSummary'] = ["Error combining summaries" for _ in range(len(df))]

        # Step 6: Generate a single final news byte covering all stocks
        logger.info("Generating final news byte for all stocks.")
        try:
            final_news_byte = self.generate_final_news_byte(df)
            df['FinalNewsByte'] = [final_news_byte for _ in range(len(df))]
        except Exception as e:
            logger.error(f"Error generating final news byte: {e}")
            final_news_byte = "Error generating final news byte."
            df['FinalNewsByte'] = [final_news_byte for _ in range(len(df))]

        # Step 7: Refine the FinalNewsByte
        logger.info("Refining the FinalNewsByte.")
        try:
            df = self.refine_final_news_byte(df)
        except Exception as e:
            logger.error(f"Error refining FinalNewsByte: {e}")
            df['FinalNewsByte'] = ["Error refining FinalNewsByte" for _ in range(len(df))]

        # Step 8: Generate a concise ReelByte
        logger.info("Generating a 1-minute ReelByte for individual stocks and combined news.")
        try:
            self.generate_reel_byte(df)
            logger.info("ReelByte generated successfully.")
        except Exception as e:
            logger.error(f"Error generating ReelByte: {e}")

        # Step 9: Generate and save the plot
        logger.info("Generating and saving plot.")
        try:
            self.generate_plot(df)
            logger.info("Uploading the plot image to S3.")
            plot_url = self.upload_image_to_s3(self.plot_path, "High Gain High Volume Today")
            logger.info(f"Plot image successfully uploaded to S3. URL: {plot_url}")
        except Exception as e:
            logger.error(f"Error generating or uploading plot: {e}")
            plot_url = None

        # Step 10: Save JSON Summary Locally
        logger.info("Saving JSON summary locally.")
        try:
            self.save_summary_as_json(final_news_byte, plot_url)
        except Exception as e:
            logger.error(f"Error saving JSON summary: {e}")

        # Step 11: Save the DataFrame to CSV
        logger.info("Saving the final output CSV.")
        try:
            df.to_csv(self.output_path, index=False)
            logger.info(f"Output saved to {self.output_path}")
        except Exception as e:
            logger.error(f"Error saving the output CSV: {e}")

        # # Step 12: Post JSON to API
        # try:
        #     logger.info("Posting JSON data to API.")
        #     json_data = {  # Ensure json_data is properly prepared
        #         "created_time": get_ist_timestamp(),
        #         "title": "Top Gainers and Losers Today",
        #         "summary_text": final_news_byte,
        #         "image_url": plot_url
        #     }
        #     self.post_json_to_summary(json_data)
        #     logger.info("JSON data posted to API successfully.")
        # except Exception as e:
        #     logger.error(f"Error posting JSON data to API: {e}")






# Run the class
HighGainHighVol().run()

2025-01-02 16:53:57 - INFO - Logger initialized successfully with IST timezone.
INFO:HighGainHighVolLogger:Logger initialized successfully with IST timezone.


File logging initialized at: /content/HighGainHighVol_2025-01-02.log
Console logging initialized.


2025-01-02 16:54:00 - INFO - OpenAI client initialized successfully.
INFO:HighGainHighVolLogger:OpenAI client initialized successfully.
2025-01-02 16:54:00 - INFO - TopLosers class initialized successfully.
INFO:HighGainHighVolLogger:TopLosers class initialized successfully.
2025-01-02 16:54:00 - INFO - Input file located at: /content/HighGainHighVol.csv
INFO:HighGainHighVolLogger:Input file located at: /content/HighGainHighVol.csv
2025-01-02 16:54:00 - INFO - Starting the Top Gainers and Losers process.
INFO:HighGainHighVolLogger:Starting the Top Gainers and Losers process.
2025-01-02 16:54:00 - INFO - Selecting top 4 stocks by DailyTurnInCr.
INFO:HighGainHighVolLogger:Selecting top 4 stocks by DailyTurnInCr.
2025-01-02 16:54:00 - INFO - Generating Contextual summaries.
INFO:HighGainHighVolLogger:Generating Contextual summaries.
2025-01-02 16:54:17 - INFO - Perplexity summary generated for Bajaj Finance.
INFO:HighGainHighVolLogger:Perplexity summary generated for Bajaj Finance.
2025-0

In [1]:
# !pip install boto3
# !pip uninstall -y openai
# !pip install openai
# !pip uninstall -y httpx
# !pip install httpx
# !pip install schedule

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 3.8 MB/s eta 0:00:00
Found existing installation: openai 1.57.4
Uninstalling openai-1.57.4:
  Successfully uninstalled openai-1.57.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 11.9 MB/s eta 0:00:00
Found existing installation: httpx 0.28.1
Uninstalling httpx-0.28.1:
  Successfully uninstalled httpx-0.28.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.0 MB/s eta 0:00:00
